In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.iolib.summary2 import summary_col
import itertools
import matplotlib.pyplot as plt

In [2]:
# Read only total_tract_population column from acs_data.csv
tract_pop = pd.read_csv(
    "../data/acs_data.csv",
    usecols=["tract", "total_tract_population"],
    dtype={"total_tract_population": np.float32, "tract": str},
)
tract_pop.columns

Index(['tract', 'total_tract_population'], dtype='object')

# Run Regression 

In [3]:
# Load the data
dtype_dict = {
    "B01003_001E_adj_supply_any_avg_tract": np.float32,
    "B01003_001E_adj_supply_hr_avg_tract": np.float32,
    "B01003_001E_totcost_any_avg_tract": np.float32,
    "state": str,
    "county": str,
    "tract": str,
    "median_household_income": np.float32,
    "gini_index": np.float32,
    "educational_attainment": np.float32,
    "employment_status": np.float32,
    "housing_tenure": np.float32,
    "poverty_status": np.float32,
    "commute_time": np.float32,
    "without_health_insurance": np.float32,
    "white": np.float32,
    "black": np.float32,
    "am_indian_alaska_native": np.float32,
    "asian": np.float32,
    "other_race": np.float32,
    "hispanic_latino": np.float32,
    "density": np.float32,
    "white_share_state": np.float32,
    "black_share_state": np.float32,
    "am_indian_alaska_native_share_state": np.float32,
    "asian_share_state": np.float32,
    "other_race_share_state": np.float32,
    "hispanic_latino_share_state": np.float32,
}


data = pd.read_csv(
    "../data/acs_data_all_test.csv", dtype=dtype_dict
)  # Replace with the path to your data file

# Merge tract_pop to data
data = data.merge(tract_pop, how="left", on="tract", validate="1:1")

# Scale median_household_income by 10,000
data["median_household_income"] = data["median_household_income"] / 10000

# Scale density by 10,000
data["density"] = data["density"] / 10000

# Rename the variables
data.rename(
    columns={
        "B01003_001E_adj_supply_any_avg_tract": "Average Supply of Any Provider",
        "B01003_001E_adj_supply_hr_avg_tract": "Average Supply of HR Provider",
        "B01003_001E_totcost_any_avg_tract": "Average Total Cost of Any Provider",
        "median_household_income": "Median Household Income",
        "gini_index": "Gini Index",
        "educational_attainment": "Educational Attainment",
        "employment_status": "Share Employed",
        "housing_tenure": "Housing Tenure",
        "poverty_status": "Share in Poverty",
        "commute_time": "Commute Time",
        "without_health_insurance": "Without Health Insurance",
        "white": "Share of White Population",
        "black": "Share of Black Population",
        "am_indian_alaska_native": "Share of American Indian/Alaska Native Population",
        "asian": "Share of Asian Population",
        "other_race": "Share of Other Race Population",
        "hispanic_latino": "Share of Hispanic/Latino Population",
        "density": "Population Density",
        "tract_pop": "total_tract_population",
        "white_share_state": "Share of White Population State",
        "black_share_state": "Share of Black Population State",
        "am_indian_alaska_native_share_state": "Share of American Indian/Alaska Native Population State",
        "asian_share_state": "Share of Asian Population State",
        "other_race_share_state": "Share of Other Race Population State",
        "hispanic_latino_share_state": "Share of Hispanic/Latino Population State",
    },
    inplace=True,
)

# calculate state population
state_population = data.groupby("state")["total_tract_population"].transform("sum")

# create tract_weight
data["tract_weight"] = data["total_tract_population"] / state_population

In [4]:
data[["tract","Share of White Population", "Share of Asian Population"]].head()

,tract,Share of White Population,Share of Asian Population
0,27001770100,0.935268,0.007589
1,27001770200,0.899737,0.001751
2,27001770300,0.933104,0.003732
3,27001770400,0.919192,0.000000
4,27001790501,0.977953,0.007349


In [5]:
data.head()

,Average Supply of Any Provider,Average Supply of HR Provider,Average Total Cost of Any Provider,state,county,tract,Median Household Income,Gini Index,Educational Attainment,Share Employed,...,Share of Other Race Population,Share of Hispanic/Latino Population,Population Density,Share of Black Population State,Share of American Indian/Alaska Native Population State,Share of Asian Population State,Share of Other Race Population State,Share of Hispanic/Latino Population State,total_tract_population,tract_weight
0,0.024394,0.007030,140.768204,27,27001,27001770100,4.6548,0.4291,0.142012,0.455980,...,0.011161,0.020089,0.000218,0.000043,0.000821,0.000063,0.000165,0.000150,2288.0,0.000426
1,0.277115,0.000000,49.739838,27,27001,27001770200,4.8580,0.3815,0.084266,0.442195,...,0.046410,0.008757,0.000226,0.000020,0.001756,0.000015,0.000702,0.000067,2014.0,0.000375
2,0.481750,0.081783,147.445908,27,27001,27001770300,4.4543,0.4515,0.155082,0.498758,...,0.023256,0.020672,0.004462,0.000080,0.000744,0.000048,0.000536,0.000240,3544.0,0.000660
3,0.420769,0.028768,91.856232,27,27001,27001770400,4.8708,0.4339,0.121503,0.485145,...,0.015488,0.013468,0.000295,0.000043,0.002652,0.000000,0.000304,0.000134,2972.0,0.000553
4,0.009878,0.000000,59.393177,27,27001,27001790501,5.0110,0.4564,0.087353,0.438879,...,0.007349,0.005249,0.000295,0.000011,0.000000,0.000052,0.000093,0.000033,1895.0,0.000353


In [6]:
def run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=1,
):
    regression_results = {}
    rmse_results = {}

    for covariates, model_name in zip(covariates_list, model_names):
        X = sm.add_constant(data[covariates])
        y = data[outcome_variable] * scalar

        # Check if weights_variable is provided
        if weights_variable:
            # Create weights array
            weights = data[weights_variable]
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)

        result = model.fit()

        rmse = np.sqrt(mean_squared_error(y, result.predict()))

        # Use model name as a key to store results
        regression_results[model_name] = result
        rmse_results[model_name] = rmse

    return regression_results, rmse_results

### Access to Any Provider

In [7]:
# Define the covariates

# First set of covariates will include racial group shares
first_set = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
]

# Second set of covariates will include the first set plus educational attainment, median income
second_set = first_set + [
    "Median Household Income",
    "Educational Attainment",
]

# Third set of covariates will include the second set plus age, population density, and unemployment rate
third_set = second_set + [
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
]

# The fourth set of covariates will include the third set plus the population density
fourth_set = third_set + ["Population Density"]

# Combine all covariates into a list of lists
covariates_list = [first_set, second_set, third_set, fourth_set]

# Define the outcome variable
outcome_variable = "Average Supply of Any Provider"

# Define the scalar
scalar = 1  # Replace with your desired scalar

# Define model names
model_names = ["First Set", "Second Set", "Third Set", "Fourth Set"]

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [8]:
# Variables are in order
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,0.126,-0.050,-0.192*,-0.198*
,(0.094),(0.096),(0.104),(0.108)
Share of American Indian/Alaska Native Population,-0.055,-0.028,0.077,0.077
,(0.178),(0.177),(0.194),(0.194)
Share of Asian Population,-0.468***,-0.413***,-0.447***,-0.450***
,(0.124),(0.122),(0.124),(0.125)
Share of Other Race Population,0.629,0.013,-0.075,-0.086
,(0.402),(0.399),(0.400),(0.404)
Share of Hispanic/Latino Population,0.076,0.046,-0.126,-0.131
,(0.129),(0.129),(0.138),(0.142)


In [9]:
summary_df_ordered.to_csv("../data/any_regression.csv")

### High Rated Providers

In [10]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [11]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,0.271***,0.277***,0.249***,0.180***
,(0.036),(0.034),(0.036),(0.037)
Share of American Indian/Alaska Native Population,-0.136**,0.040,0.033,0.036
,(0.069),(0.062),(0.068),(0.067)
Share of Asian Population,0.161***,0.109**,0.087**,0.046
,(0.048),(0.043),(0.043),(0.043)
Share of Other Race Population,0.923***,0.446***,0.474***,0.343**
,(0.156),(0.141),(0.140),(0.139)
Share of Hispanic/Latino Population,0.102**,0.229***,0.200***,0.132***
,(0.050),(0.046),(0.048),(0.049)


In [12]:
summary_df_ordered.to_csv("../data/hr_regression.csv")

### Total Cost

In [13]:
# Define the outcome variable
outcome_variable = "Average Total Cost of Any Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [14]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,110.548***,202.666***,224.630***,209.426***
,(20.450),(14.765),(15.397),(15.970)
Share of American Indian/Alaska Native Population,-282.647***,-90.292***,-27.683,-27.082
,(38.683),(27.127),(28.755),(28.640)
Share of Asian Population,192.014***,104.786***,123.692***,114.542***
,(26.822),(18.676),(18.339),(18.461)
Share of Other Race Population,679.430***,419.512***,440.347***,411.425***
,(87.200),(61.273),(59.219),(59.587)
Share of Hispanic/Latino Population,-35.932,126.103***,132.127***,117.072***
,(28.005),(19.802),(20.492),(20.881)


In [15]:
summary_df_ordered.to_csv("../data/cost_regression.csv")

In [16]:
# Create a scatter plot of share of black population vs access to any provider
# scatter_plot(
# x, y, x_label="Variable 1", y_label="Variable 2", title="Example Scatter Plot"
# )